In [1]:
import pandas as pd
import numpy as np
from env import conn
import matplotlib as plt
%matplotlib inline
cs = conn.cursor()

In [2]:
cs.execute("SELECT * FROM METRICS.LOYALTY_CARDS.LC__LINKS_JOINS__DAILY_RETAILER_CHANNEL")

In [3]:
query_result = cs.fetchall()

In [4]:
column_names = [col[0] for col in cs.description]

In [5]:
cs.close()

True

In [6]:
df = pd.DataFrame(query_result, columns=column_names)

In [7]:
import statsmodels
from statsmodels.tsa.ar_model import AutoReg

In [8]:
df.head()

,DATE,CHANNEL,BRAND,LOYALTY_PLAN_NAME,LOYALTY_PLAN_COMPANY,LC029__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__PIT,LC031__FAILED_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__PIT,LC030__REQUESTS_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__PIT,LC032__DELETED_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__PIT,LC025__SUCCESSFUL_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__PIT,...,LC017__SUCCESSFUL_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__DCOUNT_USER,LC020__DELETED_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__DCOUNT_USER,LC073__REQUESTS_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC074__FAILED_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC075__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC076__DELETED_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC077__REQUESTS_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC078_FAILED_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC079__SUCCESSFUL_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__CSUM,LC080__DELETED_LOYALTY_CARD_LINKS__DAILY_CHANNEL_BRAND_RETAILER__CSUM
0,2023-05-13,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,4169,400,129,53,891,...,7,0,55314,3679,54353,946,18282,4218,5991,304
1,2023-08-04,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,7872,420,137,159,1532,...,3,0,73141,4207,71712,1531,24095,6960,9062,436
2,2023-09-04,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,7961,435,87,248,1521,...,0,1,73679,4281,72389,1975,24233,7027,9133,561
3,2023-09-26,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,7909,434,87,301,1511,...,0,1,73685,4287,72389,2180,24233,7027,9133,599
4,2023-09-02,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,7964,435,87,245,1522,...,0,0,73679,4281,72389,1954,24233,7027,9133,557


In [14]:
df = df.filter(items=['DATE','CHANNEL','BRAND','LOYALTY_PLAN_NAME','LOYALTY_PLAN_COMPANY','LC075__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM'], axis=1)

In [15]:
df.head()

,DATE,CHANNEL,BRAND,LOYALTY_PLAN_NAME,LOYALTY_PLAN_COMPANY,LC075__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM
0,2023-05-13,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,54353
1,2023-08-04,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,71712
2,2023-09-04,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,72389
3,2023-09-26,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,72389
4,2023-09-02,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,72389


In [16]:
n = len(df)
ntrain = int(n*0.9)
ntest = n - ntrain
lag = 10
res = AutoReg(df[:ntrain], lags = lag).fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [17]:
df.dtypes

DATE                                                                        object
CHANNEL                                                                     object
BRAND                                                                       object
LOYALTY_PLAN_NAME                                                           object
LOYALTY_PLAN_COMPANY                                                        object
LC075__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM     int64
dtype: object

In [18]:
rslt_df = df.loc[df['CHANNEL'] == 'LLOYDS'] 

In [19]:
rslt_df.head()

,DATE,CHANNEL,BRAND,LOYALTY_PLAN_NAME,LOYALTY_PLAN_COMPANY,LC075__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM
0,2023-05-13,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,54353
1,2023-08-04,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,71712
2,2023-09-04,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,72389
3,2023-09-26,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,72389
4,2023-09-02,LLOYDS,HALIFAX,Iceland Bonus Card,Iceland,72389


In [20]:
df = rslt_df.loc[df['LOYALTY_PLAN_COMPANY'] == 'Stonegate Group'] 

In [21]:
df.head()

,DATE,CHANNEL,BRAND,LOYALTY_PLAN_NAME,LOYALTY_PLAN_COMPANY,LC075__SUCCESSFUL_LOYALTY_CARD_JOINS__DAILY_CHANNEL_BRAND_RETAILER__CSUM
244,2023-10-28,LLOYDS,BOS,MiXR,Stonegate Group,4
1580,2023-11-14,LLOYDS,BOS,MiXR,Stonegate Group,10
1832,2023-11-04,LLOYDS,BOS,MiXR,Stonegate Group,9
1833,2023-11-17,LLOYDS,BOS,MiXR,Stonegate Group,10
1849,2023-11-13,LLOYDS,LLOYDS,MiXR,Stonegate Group,10


In [23]:
df = df.groupby(['CHANNEL']).sum()

TypeError: unsupported operand type(s) for +: 'datetime.date' and 'datetime.date'